In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.model_selection import validation_curve, learning_curve
import graphviz

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = "/Users/fiona/Desktop/DS-SF-42/DS-SF-42/final project/H1B Disclosure Dataset Files/1. Master H1B Dataset.csv"

h1b = pd.read_csv(path)

h1b.head()

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CASE_SUBMITTED_DAY,CASE_SUBMITTED_MONTH,CASE_SUBMITTED_YEAR,DECISION_DAY,DECISION_MONTH,DECISION_YEAR,VISA_CLASS,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_COUNTRY,...,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE,CASE_STATUS
0,24,2,2016,1,10,2016,H1B,DISCOVER PRODUCTS INC,IL,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,65811.0,67320.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN
1,4,3,2016,1,10,2016,H1B,DFS SERVICES LLC,IL,UNITED STATES OF AMERICA,...,2015.0,WILLIS TOWERS WATSON SURVEY,53000.0,57200.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN
2,10,3,2016,1,10,2016,H1B,EASTBANC TECHNOLOGIES LLC,DC,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,77000.0,0.0,Year,Y,N,DC,20007,CERTIFIEDWITHDRAWN
3,28,9,2016,1,10,2016,H1B,INFO SERVICES LLC,MI,UNITED STATES OF AMERICA,...,2016.0,OFLC ONLINE DATA CENTER,102000.0,0.0,Year,Y,N,NJ,7302,WITHDRAWN
4,22,2,2015,2,10,2016,H1B,BBandT CORPORATION,NC,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,132500.0,0.0,Year,N,N,NY,10036,CERTIFIEDWITHDRAWN


In [3]:
h1b.shape

(528134, 27)

In [4]:
# make columns lower case
h1b.columns = map(str.lower, h1b.columns)

In [5]:
# check unique values
h1b.case_status.unique()

array(['CERTIFIEDWITHDRAWN', 'WITHDRAWN', 'CERTIFIED', 'DENIED'], dtype=object)

In [6]:
h1b.visa_class.unique()

array(['H1B', 'E3 Australian', 'H1B1 Chile', 'H1B1 Singapore'], dtype=object)

In [7]:
h1b.columns

Index([u'case_submitted_day', u'case_submitted_month', u'case_submitted_year',
       u'decision_day', u'decision_month', u'decision_year', u'visa_class',
       u'employer_name', u'employer_state', u'employer_country', u'soc_name',
       u'naics_code', u'total_workers', u'full_time_position',
       u'prevailing_wage', u'pw_unit_of_pay', u'pw_source', u'pw_source_year',
       u'pw_source_other', u'wage_rate_of_pay_from', u'wage_rate_of_pay_to',
       u'wage_unit_of_pay', u'h-1b_dependent', u'willful_violator',
       u'worksite_state', u'worksite_postal_code', u'case_status'],
      dtype='object')

In [8]:
# see all kinds of job titles
job_titles = h1b.groupby(['soc_name'])['case_status'].count()
job_titles_df = pd.DataFrame(job_titles)
job_titles_df.columns = ['counts']
job_titles_df.sort_values(by = ['counts'], ascending= False)

,counts
soc_name,
COMPUTER OCCUPATION,245883
ANALYSTS,112776
ENGINEERS,43843
SCIENTIST,17939
FINANCE,13199
DOCTORS,13079
EDUCATION,12371
MARKETING,10651
ACCOUNTANTS,10589


In [9]:
#look at wage unit
h1b.wage_unit_of_pay.value_counts()

Year         494664
Hour          32793
Month           397
Week            172
Bi-Weekly       104
Name: wage_unit_of_pay, dtype: int64

In [10]:
#look at wage unit
h1b.pw_unit_of_pay.value_counts()

Year         494565
Hour          33103
Month           263
Week            123
Bi-Weekly        47
Name: pw_unit_of_pay, dtype: int64

^ need to transform data into all annual pay

In [11]:
#transform data into annual pay
h1b_1 = h1b.loc[h1b["wage_unit_of_pay"] == 'Year']
h1b_1["annual_wage"] = h1b["wage_rate_of_pay_from"]
h1b_1

h1b_2 = h1b.loc[h1b["wage_unit_of_pay"] == 'Hour']
h1b_2["annual_wage"] = h1b["wage_rate_of_pay_from"] * 8760
h1b_2

h1b_3 = h1b.loc[h1b["wage_unit_of_pay"] == 'Month']
h1b_3["annual_wage"] = h1b["wage_rate_of_pay_from"] * 12
h1b_3

h1b_4 = h1b.loc[h1b["wage_unit_of_pay"] == 'Week']
h1b_4["annual_wage"] = h1b["wage_rate_of_pay_from"] * 52
h1b_4

h1b_5 = h1b.loc[h1b["wage_unit_of_pay"] == 'Bi-Weekly']
h1b_5["annual_wage"] = h1b["wage_rate_of_pay_from"] * 26
h1b_5


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,case_submitted_day,case_submitted_month,case_submitted_year,decision_day,decision_month,decision_year,visa_class,employer_name,employer_state,employer_country,...,pw_source_other,wage_rate_of_pay_from,wage_rate_of_pay_to,wage_unit_of_pay,h-1b_dependent,willful_violator,worksite_state,worksite_postal_code,case_status,annual_wage
1411,29,9,2016,3,10,2016,H1B,AWS INC,TN,UNITED STATES OF AMERICA,...,NONE,2825.00,0.00,Bi-Weekly,N,N,TN,38104,DENIED,73450.00
7531,4,10,2016,11,10,2016,H1B,AWS INC,TN,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,2825.00,0.00,Bi-Weekly,N,N,TN,38104,CERTIFIED,73450.00
8160,4,10,2016,11,10,2016,H1B,MONTCLAIR STATE UNIVERSITY,NJ,UNITED STATES OF AMERICA,...,AMERICAN FEDERATION OF TEACHERS,2064.09,0.00,Bi-Weekly,N,N,NJ,7043,CERTIFIED,53666.34
10865,6,10,2016,13,10,2016,H1B,BRISTOL COMPRESSORS INTERNATIONAL INC,VA,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,3346.16,0.00,Bi-Weekly,N,N,VA,24202,CERTIFIED,87000.16
11652,6,10,2016,13,10,2016,H1B,SIOUXLAND INTERSTATE METROPOLITAN PLANNING COU...,IA,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,1461.54,0.00,Bi-Weekly,N,N,IA,51105,DENIED,38000.04
18541,13,10,2016,19,10,2016,H1B,WHITNEY MUSEUM OF AMERICAN ART,NY,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,7124.00,7338.46,Bi-Weekly,N,N,NY,10014,CERTIFIED,185224.00
19679,14,10,2016,20,10,2016,H1B,MONTCLAIR STATE UNIVERSITY,NJ,UNITED STATES OF AMERICA,...,AMERICAN FEDERATION OF TEACHERS,1952.70,0.00,Bi-Weekly,N,N,NJ,7043,CERTIFIED,50770.20
22109,18,10,2016,24,10,2016,E3 Australian,BEARDON SERVICES INC,OK,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,1645.00,1645.00,Bi-Weekly,NaN,NaN,OK,74074,CERTIFIED,42770.00
30718,30,10,2016,31,10,2016,H1B,SIOUXLAND INTERSTATE METROPOLITAN PLANNING COU...,IA,UNITED STATES OF AMERICA,...,NONE,1461.54,0.00,Bi-Weekly,N,N,IA,51105,DENIED,38000.04
39020,7,11,2016,8,11,2016,H1B,TRENDSETTERS STYLZ and CUTZ,GA,UNITED STATES OF AMERICA,...,NONE,700.00,0.00,Bi-Weekly,N,N,GA,30214,DENIED,18200.00


In [16]:
# Concatenate of two DataFrames by rows
h1b_clean = pd.concat([h1b_1, h1b_2, h1b_3, h1b_4, h1b_5])

In [17]:
h1b_clean.shape

(528130, 28)

In [18]:
h1b_clean.columns

Index([u'case_submitted_day', u'case_submitted_month', u'case_submitted_year',
       u'decision_day', u'decision_month', u'decision_year', u'visa_class',
       u'employer_name', u'employer_state', u'employer_country', u'soc_name',
       u'naics_code', u'total_workers', u'full_time_position',
       u'prevailing_wage', u'pw_unit_of_pay', u'pw_source', u'pw_source_year',
       u'pw_source_other', u'wage_rate_of_pay_from', u'wage_rate_of_pay_to',
       u'wage_unit_of_pay', u'h-1b_dependent', u'willful_violator',
       u'worksite_state', u'worksite_postal_code', u'case_status',
       u'annual_wage'],
      dtype='object')

In [19]:
# drop unnecessary features
h1b_clean = h1b_clean.drop(['prevailing_wage', 'pw_unit_of_pay','pw_source','pw_source_year'
                            ,'pw_source_other','wage_rate_of_pay_to','wage_rate_of_pay_from','wage_unit_of_pay'], axis = 1)

## Logistic Regression

In [20]:
# make dummy for categorical vars
visa_class_df = pd.get_dummies(h1b.visa_class, prefix = 'visa_class')
employer_name_df = pd.get_dummies(h1b.employer_name, prefix = 'employer_name')
employer_state_df = pd.get_dummies(h1b.employer_state, prefix = 'employer_state')
employer_country_df = pd.get_dummies(h1b.employer_country, prefix = 'employer_country')
soc_name_df = pd.get_dummies(h1b.soc_name, prefix = 'soc_name')
naics_code_df = pd.get_dummies(h1b.naics_code, prefix = 'naics_code')
full_time_position_df = pd.get_dummies(h1b.full_time_position, prefix = 'full_time_position')
h_1b_dependent_df = pd.get_dummies(h1b["h-1b_dependent"], prefix = 'h+1b_dependent')
willful_violator_df = pd.get_dummies(h1b.willful_violator, prefix = 'willful_violator')
worksite_state_df = pd.get_dummies(h1b.worksite_state, prefix = 'worksite_state')
# worksite_postal_code_df = pd.get_dummies(h1b.worksite_postal_code, prefix = 'worksite_postal_code')


In [ ]:
h1b_clean_new = pd.concat([h1b_clean, visa_class_df, employer_name_df, employer_state_df, employer_country_df, soc_name_df,
                         naics_code_df, full_time_position_df, h_1b_dependent_df, willful_violator_df, 
                         worksite_state_df], axis=1, join_axes=[h1b_clean.index])

In [ ]:
h1b_clean_new.shape

In [110]:
#check mull accuracy
h1b.case_status.value_counts(normalize=True)

CERTIFIED             0.887974
CERTIFIEDWITHDRAWN    0.068486
WITHDRAWN             0.030318
DENIED                0.013222
Name: case_status, dtype: float64

In [1]:
X = h1b_clean_new.drop(['case_status'], axis = 1)
y = hb1_clean_new.case_status
model = linear_model.LogisticRegression().\
    fit(X, y)

print model.intercept_
print model.coef_

NameError: name 'h1b_clean_new' is not defined